# Let's try out some toy data

Consider a village where all villagers are either healthy or have a fever and only the village doctor can determine whether each has a fever. The doctor diagnoses fever by asking patients how they feel. The villagers may only answer that they feel normal, dizzy, or cold.

The doctor believes that the health condition of his patients operate as a discrete Markov chain. There are two states, "Healthy" and "Fever", but the doctor cannot observe them directly; they are hidden from him. On each day, there is a certain chance that the patient will tell the doctor he/she is "normal", "cold", or "dizzy", depending on their health condition.

In [29]:
# This is the sequence of observations (analogous to the words in a sentence)
obs = ['normal', 'cold', 'dizzy']

# This is the set of hidden states that emit observations 
# (analogous to Parts of Speech, Semantic Roles, Semantic Categories, etc., in NLP)
states = ('Healthy', 'Fever')

In this piece of code, `start_p` (`p` = probability) represents the doctor's belief about which state the HMM is in when the patient first visits (all he knows is that the patient tends to be healthy). The particular probability distribution used here is not the equilibrium one, which is (given the transition probabilities) approximately `{'Healthy': 0.57, 'Fever': 0.43}`. 

The `transition_p` represents the change of the health condition in the underlying Markov chain. In this example, there is only a 30% chance that tomorrow the patient will have a fever if he is healthy today.

The `emission_p` represents how likely the patient is to feel on each day. If he is healthy, there is a 50% chance that he feels normal; if he has a fever, there is a 60% chance that he feels dizzy.

In [ ]:
# This is the probility of starting the sequence in each state
start_p = {
    'Healthy': 0.6, 
    'Fever': 0.4
}

# This is the probability of transitioning between states at each time step
trans_p = {
   'Healthy' : {'Healthy': 0.7, 'Fever': 0.3},
   'Fever' : {'Healthy': 0.4, 'Fever': 0.6}
   }

# This is the probability of each state emitting each observation
emit_p = {
   'Healthy' : {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
   'Fever' : {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6}
   }

# We can visualize these start, transition, and emission probabilities graphically
<img src="https://upload.wikimedia.org/wikipedia/commons/0/0c/An_example_of_HMM.png" width=400>
Image credit: By Reelsun - By using open office draw, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=19118190

# Let's answer our original question using the Viterbi algorithm!

The function viterbi takes the following arguments: obs is the sequence of observations, e.g. `['normal', 'cold', 'dizzy']`; states is the set of hidden states; `start_p` is the start probability; `trans_p` are the transition probabilities; and `emit_p` are the emission probabilities. For simplicity of code, we assume that the observation sequence obs is non-empty and that `trans_p[i][j]` and `emit_p[i][j]` is defined for all states i,j.

In [30]:
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]
    for st in states:
        V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}
    # Run Viterbi when t > 0
    for t in range(1, len(obs)):
        V.append({})
        for st in states:
            max_tr_prob = max(V[t-1][prev_st]["prob"]*trans_p[prev_st][st] for prev_st in states)
            for prev_st in states:
                if V[t-1][prev_st]["prob"] * trans_p[prev_st][st] == max_tr_prob:
                    max_prob = max_tr_prob * emit_p[st][obs[t]]
                    V[t][st] = {"prob": max_prob, "prev": prev_st}
                    break
    for line in dptable(V):
        print(line)
    opt = []
    # The highest probability
    max_prob = max(value["prob"] for value in V[-1].values())
    previous = None
    # Get most probable state and its backtrack
    for st, data in V[-1].items():
        if data["prob"] == max_prob:
            opt.append(st)
            previous = st
            break
    # Follow the backtrack till the first observation
    for t in range(len(V) - 2, -1, -1):
        opt.insert(0, V[t + 1][previous]["prev"])
        previous = V[t + 1][previous]["prev"]
 
    print('The steps of states are ' + ' '.join(opt) + ' with highest probability of %s' % max_prob)

def dptable(V):
     # Print a table of steps from dictionary
     yield " ".join(("%12d" % i) for i in range(len(V)))
     for state in V[0]:
         yield "%.7s: " % state + " ".join("%.7s" % ("%f" % v[state]["prob"]) for v in V)

In [32]:
viterbi(obs, states, start_p, trans_p, emit_p)

           0            1            2
Healthy: 0.30000 0.08400 0.00588
Fever: 0.04000 0.02700 0.01512
The steps of states are Healthy Healthy Fever with highest probability of 0.01512


# Let's try building a POS tagger!

We've included a file `pos-training-data.tsv` that contains POS-tagged data from Twitter.  The file is formatted as two columns where column 1 is the word and column 2 is the part of speech.  Sentences are separated by a blank line, which you'll need to take into account when computing the starting transition probabilities.

Your job is to read the data and then fill the `states`, `start_p`, `trans_p`, and `emit_p` data structures based on the observed word and POS transitions.  Once those are done, you can run the viterbi algorithm as-is!  We recommend trying this process in two steps
1. First, *count* the frequencies for the start, transition, and emission (Hint: `defaultdict` and `Counter` are handy here!)
2. Then, *normalize* the counts to be probability distributions (e.g., convert `start_counts` to `start_p`)

Side note:  The date comes from the paper <a href="http://www.ark.cs.cmu.edu/TweetNLP/gimpel+etal.acl11.pdf ">Part-of-Speech Tagging for Twitter: Annotation, Features, and Experiments</a> by Kevin Gimpel, Nathan Schneider, Brendan O'Connor, Dipanjan Das, Daniel Mills,   Jacob Eisenstein, Michael Heilman, Dani Yogatama, Jeffrey Flanigan, and  Noah A. Smith, iun the Proceedings of ACL 2011.   Also, here's their project's <a href="https://github.com/brendano/ark-tweet-nlp/">website</a> and the <a href="http://www.cs.cmu.edu/~ark/TweetNLP/annot_guidelines.pdf">part of speech guidelines</a> (you don't really need to read these).

In [20]:
from collections import *

with open('pos-training-data.tsv', encoding='utf-8') as f:
       
    for line_no, line in enumerate(f):

        # Do your counting thing here!

In [ ]:
# Now normalize the counts

# Let's test it out!
Once you've finished your code, try it out here states, start_p, trans_p, emit_p

In [ ]:
viterbi(['I', 'like', 'monkeys'], states, start_p, trans_p, emit_p)
viterbi(['Just', 'finished', 'my', 'morning', 'Zumba', 'class'], states, start_p, trans_p, emit_p)
viterbi(['I', 'am', 'so', 'bored', 'right', 'now'], states, start_p, trans_p, emit_p)
viterbi(['I', 'luv', 'youuuuuu'], states, start_p, trans_p, emit_p)

# Have even more time?
We've also included `pos-test-data.tsv`.  For fun, try loading these instances and see how well your Viterbi-based POS tagger works!